In [1]:
#Train model on kaggle dataset
#import libraries
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import numpy as np
import nltk
import pickle
import sqlite3 as sql
pd.options.display.max_colwidth = 1000

In [14]:
df = pd.read_csv('Tweets.csv') #from kaggle
df
lst = []


In [15]:
#df #predicting the airline_sentiment
#remove twitter handles(@user)

def remove_pattern(input_txt, pattern):  
    r =  re.findall(pattern, input_txt)
    for word in r:
         input_txt = re.sub(word, "", input_txt)
    return input_txt
df['clean_text'] = np.vectorize(remove_pattern)(df['text'], '@')#remove twitter(@user)

df

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),VirginAmerica What dhepburn said.
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),VirginAmerica plus you've added commercials to the experience... tacky.
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),VirginAmerica I didn't today... Must mean I need to take another trip!
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),"VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse"
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing about it,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),VirginAmerica and it's a really big bad thing about it
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different flight to Chicago.,NaN,2015-02-22 12:01:01 -0800,NaN,NaN,AmericanAir thank you we got on a different flight to Chicago.
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flight. No warnings or communication until we were 15 minutes Late Flight. That's called shitty customer svc,NaN,2015-02-22 11:59:46 -0800,Texas,NaN,AmericanAir leaving over 20 minutes Late Flight. No warnings or communication until we were 15 minutes Late Flight. That's called shitty customer svc
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to #BlackBerry10,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN,AmericanAir Please bring American Airlines to #BlackBerry10
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada),"AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??"


In [16]:
#remove special, characters, numbers and punctations

df['clean_text'] = df['clean_text'].str.replace("[^a-zA-Z#]", " ")


In [17]:
#remove short words

df['clean_text'] = df['clean_text'].apply(lambda x: " ".join([w for w in x.split() if len(w) > 3]))

df

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),VirginAmerica What dhepburn said
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),VirginAmerica plus added commercials experience tacky
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),VirginAmerica didn today Must mean need take another trip
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),VirginAmerica really aggressive blast obnoxious entertainment your guests faces they have little recourse
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing about it,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),VirginAmerica really thing about
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different flight to Chicago.,NaN,2015-02-22 12:01:01 -0800,NaN,NaN,AmericanAir thank different flight Chicago
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flight. No warnings or communication until we were 15 minutes Late Flight. That's called shitty customer svc,NaN,2015-02-22 11:59:46 -0800,Texas,NaN,AmericanAir leaving over minutes Late Flight warnings communication until were minutes Late Flight That called shitty customer
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to #BlackBerry10,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN,AmericanAir Please bring American Airlines #BlackBerry
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada),AmericanAir have money change flight answer your phones other suggestions make commitment


In [18]:
tokenized_text = df['clean_text'].apply(lambda x: x.split())
tokenized_text.head

#stem the words

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

tokenized_text = tokenized_text.apply(lambda sentence: [stemmer.stem(word) for word in sentence])

tokenized_text.head()

for i in range(len(tokenized_text)):
    tokenized_text[i] = " ".join(tokenized_text[i])

df['clean_text'] = tokenized_text
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),virginamerica what dhepburn said
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),virginamerica plu ad commerci experi tacki
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),virginamerica didn today must mean need take anoth trip
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),virginamerica realli aggress blast obnoxi entertain your guest face they have littl recours
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing about it,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),virginamerica realli thing about


In [19]:
# feature extraction

from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_df = 0.90, max_features =500, stop_words='english')

bow = bow_vectorizer.fit_transform(df['clean_text'])


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
x_train, x_test, y_train, y_test = train_test_split(bow, df['airline_sentiment'], random_state=42, test_size=0.20)
#training
model = LogisticRegression()
model.fit(x_train, y_train)

C:\Users\david\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [21]:
#test set accuracy
pred = model.predict(x_test)
accuracy_score(y_test, pred)

0.7762978142076503

In [22]:
#training set accuracy

model.score(x_train, y_train)

0.7945696721311475

In [11]:
#x-axis response time
#y-axis change sentiment
#Specific on KLM

#x-axis airlines
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
def applythis(x):
    u = list()
    try:
        for w in x.split():
            if len(w) > 3:
                u.append(w+' ')
    except:
        return ''
    return ''.join(u)

def applysplit(sentence):
    words = sentence.split(' ')
    for num,i in enumerate(words):
        words[num] = stemmer.stem(i)
    return ' '.join(words)
                

def cleantext(textseries):
    returnseries = textseries.str.replace("[^a-zA-z#]", " ")
    returnseries = returnseries.apply(lambda x: applythis(x))
    returnseries = returnseries.apply(lambda x: x.lower())
    returnseries = returnseries.apply(lambda x: applysplit(x))
    return returnseries

In [3]:
conn = sql.connect('Tweets5.db')

In [4]:
tweetframe = pd.read_sql_query('SELECT in_reply_to_status_id, created_at, id, text, user_id FROM tweets',conn)

In [12]:
tweetframe['cleantexttext'] = cleantext(tweetframe['text'])

In [23]:
sentilist = model.predict_proba(bow_vectorizer.fit_transform(tweetframe['cleantexttext'].to_list()))

In [24]:
sentiscaled = []
for num, i in enumerate(sentilist):
    sentiscaled.append(np.sum(i*np.array([-1,0,1])))

In [ ]:
tweetframe = tweetframe.drop('cleantexttext',axis=1)
sentidict = dict(zip(tweetframe['id'],tweetframe['sentiment']))
tweetframe.to_csv('AllTweetsAnalysed.csv')
del tweetframe

In [139]:
with open('conversations.pkl','rb') as input:
    conversationlist1 = pickle.load(input)

In [33]:
conversationlist = list()

for i in conversationlist1:
    i = pd.DataFrame(i)
    for index,j in i.iterrows():
        i.loc[index,'sentiment'] = sentidict[j['id']]
        newframe = newframe.append(j)
        
    conversationlist.append(newframe.sort_values(by='created_at'))

In [1]:
for num,i in enumerate(conversationlist):
    miniconv = i
    conversationlist[num] = {'conv': i,'delta': miniconv.loc[miniconv.shape[0]-1,'sentiment']-miniconv.loc[0,'sentiment'],'mean':miniconv['sentiment'].mean(), 'time': miniconv.loc[0,'created_at']}

NameError: name 'conversationlist' is not defined

mminierlist[0

In [141]:
conversationframe = pd.DataFrame(conversationcopy).sort_values(by=['time'])
conversationframe = conversationframe.set_index('time')

In [123]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
save_object(conversationframe,'conversationframe.pkl')